# `tf.data`: la mejor manera de gestionar datos en TensorFlow

Tensorflow es una de las librerías más usadas para la implementación de modelos de Deep Learning, como las redes neuronales, las convolucionales, las lstm o las redes transformer.

Y una fase clave en esta implementación es lograr llevar los datos desde la fuente, que puede ser una base de datos o una carpeta de archivos, hasta el modelo para poder realizar su entrenamiento.

Así que en este tutorial veremos una introducción al módulo `tf.data` de TensorFlow, el método más recomendado para realizar esta gestión de los datos a través de esta librería.

Entonces comenzaremos entendiendo las principales características y ventajas de `tf.data` frente a otras alternativas y luego veremos un ejemplo básico de uso para entender la lógica de funcionamiento de este módulo.

## 1. Creación de un dataset

In [ ]:
# Importar TensorFlow
import tensorflow as tf

En este tutorial introductorio veremos las herramientas básicas que ofrece `tf.data`.

Así que usaremos un dataset que consiste en un simple arreglo de números y veremos cómo gestionarlo usando este módulo.

En próximos tutoriales veremos formas más avanzadas de usar el módulo usando datos como imágenes, texto o datos tabulares.

In [ ]:
# Crear un dataset super-simple
x = [23, 72, 17, -2, 45, 1]

# Crear el dataset usando el módulo "Dataset" de "tf.data"
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

Cada dato del set de datos original es almacenado en un Tensor (un tipo de dato de TensorFlow que es en esencia un arreglo).

Para ver el contenido de este set de datos podemos iterar sobre cada elemento:

In [ ]:
# ¿Cómo ver el contenido?
for dato in dataset:
    print(dato)

tf.Tensor(23, shape=(), dtype=int32)
tf.Tensor(72, shape=(), dtype=int32)
tf.Tensor(17, shape=(), dtype=int32)
tf.Tensor(-2, shape=(), dtype=int32)
tf.Tensor(45, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)


In [ ]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

Y lo anterior nos permite verificar los detalles de cada uno de los tensores que conforman el dataset.

Podemos representar cada dato en formato NumPy para entender fácilmente su contenido:

In [ ]:
# ¿Y si quiero cada dato en formato NumPy?
for dato in dataset:
    print(dato.numpy())

23
72
17
-2
45
1


## 2. Técnicas básicas para manipular el dataset

Una primera técnica es tomar algunos elementos del dataset.

Para ello usamos el método `take` que permite tomar los "n" primeros elementos (donde "n" es el argumento de este método):

In [ ]:
# Tomar algunos elementos: take

# Los 3 primeros
for dato in dataset.take(3):
    print(dato.numpy())

23
72
17


Otra técnica muy usada es la transformación de los datos: modificar sus valores para que resulten adecuados al momento de llevarlos al modelo.

En este caso particular supongamos que no nos interesa presentar al modelo los valores negativos.

Entonces podemos usar el método `filter` combinado con una función `lambda` de Python para filtrar los elementos no deseados:

In [ ]:
# Ejemplo de transformación: filtrar los valores negativos

dataset_filtrado = dataset.filter(lambda y: y>0)

for dato in dataset_filtrado:
    print(dato.numpy())

23
72
17
45
1


O por ejemplo, supongamos que nos interesa escalar los valores por un factor de 10.

En este caso podemos usar `map` + `lambda`:

In [ ]:
# Dividir cada valor entre 10 y convertir a float
dataset_mapeado = dataset.map(lambda y: y/10)

for dato in dataset_mapeado:
    print(dato.numpy())

dataset_mapeado

2.3
7.2
1.7
-0.2
4.5
0.1


<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.float64, name=None)>

Y otra herramienta que **siempre** tendremos que usar será la mezcla aleatoria de los datos.

Esta transformación se logra con el método `shuffle`:

In [ ]:
# Mezclar los datos
dataset_mezclado = dataset.shuffle(1)

# Imprimir el dato original y el dato después de la mezcla
for dato, dato_s in zip(dataset,dataset_mezclado):
    print(dato.numpy(), dato_s.numpy())

23 23
72 72
17 17
-2 -2
45 45
1 1


En el caso anterior hemos usado un argumento igual a 1 en la función `shuffle`.

Este argumento indica la cantidad de elementos que `shuffle` almacenará en memoria al momento de hacer la mezcla aleatoria. **Esto se hace para permitir que podamos mezclar aleatoriamente sets de datos gigantescos, que tienen un tamaño mayor al de la memoria disponible**.

Entonces, en cada iteración `shuffle` tomará ese número de elementos, lo mezclará aleatoriamente y repetirá el proceso hasta agotar todos los elementos.

Por eso, con un argumento igual a 1 obtendremos exactamente el mismo arreglo original.

Y entre más alto sea el valor del argumento más aleatoria será la mezcla.

Como en nuestro caso tenemos tan sólo 6 elementos podemos tomarlos todos al momento de hacer la mezcla con `shuffle`:

In [ ]:
# Mezclar los datos
dataset_mezclado = dataset.shuffle(6)

# E imprimir el dato original y el dato mezclado
for dato, dato_s in zip(dataset,dataset_mezclado):
    print(dato.numpy(), dato_s.numpy())

23 72
72 17
17 -2
-2 45
45 1
1 23


Y como lo vimos en otro tutorial, para asegurar la reproducibilidad del entrenamiento tenemos que fijar la semilla del generador aleatorio.

En este caso la semilla permitirá obtener siempre la misma mezcla independientemente de cuando ejecutemos el código.

Esta semilla la fijamos con el argumento `seed`:

In [ ]:
# Mezclar dataset con una semilla
dataset_mezclado = dataset.shuffle(6, seed=31)

# Imprimir datos originales y mezclados
for dato, dato_s in zip(dataset,dataset_mezclado):
    print(dato.numpy(), dato_s.numpy())

23 17
72 -2
17 45
-2 1
45 23
1 72


Otra técnica que siempre tendremos que usar es la generación de lotes.

Cuando tenemos un dataset muy grande no podemos leer todos los datos en la RAM, así que tendremos que tomar bloques de datos: los lotes.

En este caso podemos usar `batch` que toma bloques de tamaño "n" en orden consecutivo.

Por ejemplo, creemos lotes de 4 elementos a partir del dataset original:

In [ ]:
# Crear batches
dataset_batches = dataset.batch(4) # Crear lotes con 4 elementos c/u

# Imprimirlos en pantalla
for batch in dataset_batches:
    print(batch)

tf.Tensor([23 72 17 -2], shape=(4,), dtype=int32)
tf.Tensor([45  1], shape=(2,), dtype=int32)


Y finalmente otra de las operaciones que comúnmente tendremos que realizar es la partición del dataset en entrenamiento, validación y prueba.

Comencemos creando un set de datos con que será un simple arreglo de 10 elementos: los números del 1 al 10:

In [ ]:
# 1. Crear el set de datos
ds = tf.data.Dataset.from_tensor_slices(range(1,11))

for dato in ds:
    print(dato.numpy())

1
2
3
4
5
6
7
8
9
10


Ahora supongamos que queremos partirlo en 60% para entrenamiento (6 datos), 20% para validación (2 datos) y 20% para prueba (también 2 datos).

Comencemos calculando estos tamaños de forma automática:

In [ ]:
N = len(ds) # Cantidad total de datos (10)
tr_size = int(0.6*N)
ts_size = int(0.2*N)
vl_size = N - tr_size - ts_size # Para que no se pierdan datos por el redondeo

print(N, tr_size, ts_size, vl_size)

10 6 2 2


Antes de hacer la partición tenemos que mezclar los datos aleatoriamente usando `shuffle`.

Acá es clave que cada vez que hagamos un llamado al dataset mezclado no se mezcle nuevamente. Esto lo hacemos fijando el argumento `reshuffle_each_iteration` como `False`:

In [ ]:
# Mezclar el dataset y desactivar opción "reshuffle_each_iteration"
SEED = 423
ds = ds.shuffle(N, seed=SEED, reshuffle_each_iteration=False)
for dato in ds:
    print(dato.numpy())

1
6
4
3
5
2
7
8
9
10


Y por último, creamos los sets de entrenamiento, validación y prueba usando los métodos `take` (tomar los primeros "n" elementos del dataset) y `skip` (tomar un dataset pero "saltarse" los primeros "n" elementos:

In [ ]:
# Entrenamiento: tomar los primeros "tr_size" elementos del dataset mezclado
ds_train = ds.take(tr_size)

# Prueba: "saltar" los primeros "tr_size" elementos (con "skip") y
# luego tomar los "ts_size" elementos restantes (con "take")
ds_test = ds.skip(tr_size).take(ts_size)

# Validación: "saltar" los primeros "tr_size+ts_size" elementos (con "skip") y
# tomar el resto
ds_val = ds.skip(tr_size + ts_size)

# Imprimir datasets
print('Train:')
for dato in ds_train:
    print(dato.numpy())
print('Test:')
for dato in ds_test:
    print(dato.numpy())
print('Val:')
for dato in ds_val:
    print(dato.numpy())

Train:
7
3
6
2
1
4
Test:
5
10
Val:
8
9


¡Y listo, estas son las herramientas básicas que debemos tener en cuenta al momento de usar `tf.data`!

## Conclusión

Acabamos de ver los principios básicos de funcionamiento del módulo `tf.data` de TensorFlow y en particular de la clase `Dataset` que permiten gestionar los datos al momento de alimentar un modelo de Deep Learning.

Y esta gestión tiene básicamente tres fases: la extracción, transformación y carga de los datos. Y precisamente el módulo `tf.data` nos permite implementar fácilmente todas estas fases independientemente del tipo de datos que tengamos (si son tablas, si son imágenes, audios, texto, etc.).

Así que el uso de `tf.data` es la forma más adecuada de gestionar los datos cuando estamos implementando modelos en TensorFlow y en próximos tutoriales veremos de forma detallada como manipular diferentes tipos de datos usando precisamente este módulo.